In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import KDTree
from sklearn.neighbors import BallTree
from sklearn.linear_model import LinearRegression
from scipy.interpolate import make_interp_spline
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv("Data/train.csv")
df_test = pd.read_csv('Data/test.csv')
df_hawk = pd.read_csv('Data/auxiliary-data/sg-gov-hawkers.csv')
df_hdb = pd.read_csv('Data/auxiliary-data/sg-hdb-block-details.csv')
df_mrt = pd.read_csv('Data/auxiliary-data/sg-mrt-stations.csv')
df_ps = pd.read_csv('Data/auxiliary-data/sg-primary-schools.csv')
df_ss = pd.read_csv('Data/auxiliary-data/sg-secondary-schools.csv')
df_mall = pd.read_csv('Data/auxiliary-data/sg-shopping-malls.csv')
df_child = pd.read_csv('Data/auxiliary-data/childcare.csv')
df_cc = pd.read_csv('Data/auxiliary-data/communityclubs.csv')
df_ht = pd.read_csv('Data/auxiliary-data/heritagetrees.csv')
df_lib = pd.read_csv('Data/auxiliary-data/libraries.csv')
df_hosp = pd.read_csv('Data/auxiliary-data/moh_hospitals.csv')
df_park = pd.read_csv('Data/auxiliary-data/nationalparks.csv')
df_pharma = pd.read_csv('Data/auxiliary-data/registered_pharmacy.csv')

# DATA CLEANING

In [ ]:
def clean_train_test_dataframe(df, is_test=False):
    def floor_range_avg(rng):
        upper, lower = rng.split(" to ")
        upper = int(upper)
        lower = int(lower)
        return (upper + lower)/2

    # Standardize the attributes BLOCK and STREET by converting to lowercase.
    df['BLOCK'] = df['BLOCK'].str.strip().str.lower()
    df['STREET'] = df['STREET'].str.strip().str.lower()

    # Add a new attribute called ADDRESS, which is a concatenation between BLOCK and STREET.
    df["ADDRESS"] = df['BLOCK'] + " " + df['STREET']

    # Standardize the FLAT_TYPE attribute by replacing hyphens (-) with spaces and converting all text to lowercase.
    df['FLAT_TYPE'] = df['FLAT_TYPE'].str.replace('-', ' ').str.strip().str.lower()

    # Extract the month & year component from the MONTH attribute, and save the month in MONTH and year in a new attribute called YEAR.
    df['YEAR'] = df['MONTH'].str.split('-').str[0].astype(int)
    df['MONTH'] = df['MONTH'].str.split('-').str[1].astype(int)

    # Convert FLOOR_RANGE to a single number by taking the average of the range.
    df["FLOOR"] = df["FLOOR_RANGE"].apply(floor_range_avg)
    df["FLAT_AGE"] = df["YEAR"] - df["LEASE_COMMENCE_DATA"]

    # Drop attributes BLOCK, STREET, FLOOR_RANGE & ECO_CATEGORY. ADDRESS contains BLOCK and STREET and is unique, FLOOR has FLOOR_RANGE so redundant info, and ECO_CATEGORY has only one value.
    df = df.drop('ECO_CATEGORY', axis=1)
    df = df.drop('FLOOR_RANGE', axis=1)
    df = df.drop('BLOCK', axis=1)
    df = df.drop('STREET', axis=1)

    # Drop duplicate rows after the above cleaning is performed.
    if not is_test:
        df.drop_duplicates(inplace=True)

    return df

In [ ]:
def clean_df_hdb(df):
    df["ADDRESS"] = df["ADDRESS"].str.strip().str.lower()
    df["BLOCK"] = df["BLOCK"].str.strip().str.lower()

    df["ADDRESS"] = df["BLOCK"] + " " + df["ADDRESS"]

    return df.drop('BLOCK', axis=1)

def remove_near_duplicate_schools(df):
    df = df.drop_duplicates(subset=['LATITUDE', 'LONGITUDE'], keep='last')
    return df

In [ ]:
df_train = clean_train_test_dataframe(df_train)
df_test = clean_train_test_dataframe(df_test, is_test=True)

df_hdb = clean_df_hdb(df_hdb)

# DATA AUGMENTATION

In [ ]:
def convert_to_coordinates(df, df_coords):
    df = df.merge(
        df_coords[['ADDRESS', 'LATITUDE', 'LONGITUDE', 'MAX_FLOOR']],
        on=['ADDRESS'],
        how='left'
    )
    return df

def calculate_distance_and_metadata(feature, df_features, df_main, meta_cols=None):
    tree = BallTree(np.radians(df_features[['LATITUDE', 'LONGITUDE']].to_numpy()), metric='haversine')
    distances, indices = tree.query(np.radians(df_main[['LATITUDE', 'LONGITUDE']].to_numpy()), k=10)
    distances_km = distances * 6371

    df_main[f"DIST_AVG_{feature}"] = np.mean(distances_km, axis=1)

    if meta_cols:
        nearest_idx = indices[:, 0]
        for col in meta_cols:
            df_main[f"{feature}_{col}"] = df_features.iloc[nearest_idx][col].values

def augment_auxiliary_data(df, mrt, shopping, hawker, ps, ss, child, cc, ht, lib, hosp, park, pharma):
    calculate_distance_and_metadata("MRT", mrt, df, meta_cols=["STATUS"])
    calculate_distance_and_metadata("OPEN_MRT", mrt[mrt["STATUS"] == "open"], df)
    calculate_distance_and_metadata("SHOPPING_MALL", shopping, df)
    calculate_distance_and_metadata(
        "HAWKER_CENTRE", hawker, df,
        meta_cols=["TYPE", "OWNER", "NUMBER_OF_STALLS"]
    )
    calculate_distance_and_metadata("PRIMARY_SCHOOL", ps, df)
    calculate_distance_and_metadata("SECONDARY_SCHOOL", ss, df)
    calculate_distance_and_metadata("CHILDCARE", child, df)
    calculate_distance_and_metadata("COMMUNITY_CLUBS", cc, df)
    calculate_distance_and_metadata("HERITAGE_TREES", ht, df)
    calculate_distance_and_metadata("LIBRARIES", lib, df)
    calculate_distance_and_metadata("HOSPITALS", hosp, df)
    calculate_distance_and_metadata("NATIONAL_PARKS", park, df)
    calculate_distance_and_metadata("PHARMACIES", pharma, df)
    return df


In [ ]:
df_train = convert_to_coordinates(df_train, df_hdb)
df_test = convert_to_coordinates(df_test, df_hdb)

df_train = augment_auxiliary_data(df_train, df_mrt, df_mall, df_hawk, df_ps, df_ss, df_child, df_cc, df_ht, df_lib, df_hosp, df_park, df_pharma)
df_test = augment_auxiliary_data(df_test, df_mrt, df_mall, df_hawk, df_ps, df_ss, df_child, df_cc, df_ht, df_lib, df_hosp, df_park, df_pharma)

In [ ]:
list(set(df_train.select_dtypes(include=['number'])).difference(set(["RESALE_PRICE"])))

['DIST_AVG_OPEN_MRT',
 'DIST_AVG_HERITAGE_TREES',
 'MAX_FLOOR',
 'YEAR',
 'DIST_AVG_HOSPITALS',
 'DIST_AVG_HAWKER_CENTRE',
 'MONTH',
 'DIST_AVG_SECONDARY_SCHOOL',
 'LEASE_COMMENCE_DATA',
 'LATITUDE',
 'DIST_AVG_NATIONAL_PARKS',
 'FLOOR',
 'FLAT_AGE',
 'DIST_AVG_PRIMARY_SCHOOL',
 'DIST_AVG_COMMUNITY_CLUBS',
 'DIST_AVG_MRT',
 'DIST_AVG_SHOPPING_MALL',
 'DIST_AVG_PHARMACIES',
 'FLOOR_AREA_SQM',
 'DIST_AVG_LIBRARIES',
 'DIST_AVG_CHILDCARE',
 'LONGITUDE',
 'HAWKER_CENTRE_NUMBER_OF_STALLS']

In [ ]:
list(set(df_train).difference(set(df_train.select_dtypes(include=['number']))))

['ADDRESS',
 'FLAT_MODEL',
 'FLAT_TYPE',
 'MRT_STATUS',
 'HAWKER_CENTRE_OWNER',
 'HAWKER_CENTRE_TYPE',
 'TOWN']

In [ ]:
target = 'RESALE_PRICE'

num_features = [
    'DIST_AVG_NATIONAL_PARKS',
    'DIST_AVG_SHOPPING_MALL',
    'DIST_AVG_PRIMARY_SCHOOL',
    'DIST_AVG_CHILDCARE',
    'DIST_AVG_HERITAGE_TREES',
    'DIST_AVG_OPEN_MRT',
    'DIST_AVG_SECONDARY_SCHOOL',
    'DIST_AVG_MRT',
    'DIST_AVG_LIBRARIES',
    'DIST_AVG_HAWKER_CENTRE',
    'DIST_AVG_PHARMACIES',
    'DIST_AVG_HOSPITALS',
    'DIST_AVG_COMMUNITY_CLUBS',
    'LATITUDE',
    'LONGITUDE',
    'YEAR',
    'MONTH',
    'FLAT_AGE',
    'FLOOR',
    'FLOOR_AREA_SQM',
    'MAX_FLOOR',
    'LEASE_COMMENCE_DATA',
    'HAWKER_CENTRE_NUMBER_OF_STALLS'
]

cat_features = [
    'HAWKER_CENTRE_TYPE',
    'HAWKER_CENTRE_OWNER',
    'FLAT_MODEL',
    'FLAT_TYPE',
    'MRT_STATUS',
    'TOWN',
]

features = cat_features + num_features

In [ ]:
X = df_train[features].copy()
y = df_train[target].copy()

In [ ]:
for col in cat_features:
     X[col] = X[col].astype("category")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# TRAINING

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
import numpy as np


param_dist = {
    'max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
    'learning_rate': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
    'min_child_weight': [1, 3, 5, 7, 9, 11],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'reg_lambda': [1, 3, 5, 7, 10],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'num_leaves': [15, 31, 63, 127, 255]
}


model = LGBMRegressor(
    n_estimators=2000,
    objective='regression',
    metric='rmse',
    random_state=42,
)


search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    scoring='neg_root_mean_squared_error',
    cv=3,
    n_iter=30,
    random_state=42,
    n_jobs=-1
)

search.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
)

print("Best parameters found:")
print(search.best_params_)
print("Best RMSE:", np.sqrt(-search.best_score_))

best_params = search.best_params_

model = LGBMRegressor(
    **best_params,
    n_estimators=2000,
    objective='regression',
    metric='rmse',
    random_state=42,
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

rmse_scorer = make_scorer(
    lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    greater_is_better=False
)

cv_scores = cross_val_score(model, X, y, scoring=rmse_scorer, cv=kf, n_jobs=-1, verbose=1)

mean_rmse = -np.mean(cv_scores)
std_rmse = np.std(cv_scores)

print("\n5-Fold Cross-Validation Results:")
print(f"RMSE scores (per fold): {-cv_scores}")
print(f"Mean RMSE: {mean_rmse:.4f}")
print(f"Std RMSE: {std_rmse:.4f}")


final_model = LGBMRegressor(
    **best_params,
    n_estimators=2000,
    objective='regression',
    metric='rmse',
    random_state=42,
)

final_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_metric='rmse',
)

y_pred = final_model.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print("\nFinal Model Evaluation (Validation Set):")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.084851 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4324
[LightGBM] [Info] Number of data points in the train set: 146217, number of used features: 29
[LightGBM] [Info] Start training from score 518926.422154
Best parameters found:
{'subsample': 0.8, 'reg_lambda': 7, 'reg_alpha': 0.1, 'num_leaves': 31, 'min_child_weight': 7, 'max_depth': 10, 'learning_rate': 0.1, 'colsample_bytree': 0.5}
Best RMSE: 159.780742179702


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished



5-Fold Cross-Validation Results:
RMSE scores (per fold): [25209.22975427 25140.89349441 24942.70691788 25202.39770792
 25080.81892573]
Mean RMSE: 25115.2094
Std RMSE: 98.0215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011702 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4324
[LightGBM] [Info] Number of data points in the train set: 146217, number of used features: 29
[LightGBM] [Info] Start training from score 518926.422154

Final Model Evaluation (Validation Set):
Mean Squared Error: 597057278.3321
Mean Absolute Error: 17740.9649
Root Mean Squared Error: 24434.7555
R² Score: 0.9819


In [ ]:
X_test = df_test[features].copy()

for col in cat_features:
     X_test[col] = X_test[col].astype("category")

y_test_pred = final_model.predict(X_test)

In [ ]:
submission = pd.DataFrame({
    "Id": df_test.index,
    "Predicted": y_test_pred
})

submission.to_csv("Data/submission_lgbm.csv", index=False)
print("Saved predictions to submission_lgbm.csv")

Saved predictions to submission_lgbm.csv
